In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .remote("sc://192.168.1.7:15002") \
  .appName("UDFTransformation") \
  .config("spark.sql.ansi.enabled", "false") \
  .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
  .getOrCreate()

# limit() shows a nice HTML table in Jupyter, while show() prints plain text  
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

spark

In [2]:
DataStreamReader = spark.readStream \
  .format("socket") \
  .option("host", "192.168.1.7") \
  .option("port", 9999)

lines = DataStreamReader.load()

DataStreamReader

In [31]:
if len(spark.streams.active) > 0:
  for query in spark.streams.active:
    query.stop()

writer = lines.writeStream.outputMode("append").format("memory").queryName("streaming_query")

query = writer.start()

In [33]:
# !just stream
# hello
# world
spark.sql("SELECT * FROM streaming_query").show()


+-----+
|value|
+-----+
|hello|
|world|
+-----+

